# Merge All DataFrames

In [1]:
import pandas as pd
import numpy as np

In [2]:
dir = '../data/'
prefix = 'FONNESBECK_'
suffix = '_20151202.csv'

In [3]:
adt_cms_final = pd.read_pickle(dir + './adt_cms_final.pkl')
adt_cms_final.head()

,ruid,visit_id,admit_date,discharge_date,hospital_day,stay_length,n_transfers,readmit_time,readmit_30d
0,50135262,0,2007-02-08,2007-02-12,2007-02-08,4 days,2,NaT,0
1,50135262,0,2007-02-08,2007-02-12,2007-02-09,4 days,2,NaT,0
2,50135262,0,2007-02-08,2007-02-12,2007-02-10,4 days,2,NaT,0
3,50135262,0,2007-02-08,2007-02-12,2007-02-11,4 days,2,NaT,0
4,50135262,0,2007-02-08,2007-02-12,2007-02-12,4 days,2,NaT,0


In [4]:
adt_cms_final.shape

(128389, 9)

In [5]:
phenotype = pd.read_pickle(dir + './phenotype.pkl')
phenotype.head()

,ruid,sex,dob,dod,race
0,50135262,F,1949-09-20,NaT,W
1,50135361,M,1932-02-15,2012-09-09,W
2,50135369,M,1958-05-04,2012-01-23,W
3,50135375,M,1943-05-01,2011-08-18,B
4,50135425,F,1946-10-02,NaT,W


In [6]:
merged = adt_cms_final.merge(phenotype, how='left', on='ruid')
merged.shape

(128389, 13)

In [7]:
cpt_wide = pd.read_pickle(dir + './cpt_wide.pkl')
cpt_wide.head()

code_cat,ruid,event_date,anesthesia,eval_manage,expired,medicine,modifier,path_lab,radiology,surgery,unknown
0,50135262,2005-01-09,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,50135262,2007-02-08,0.0,1.0,2.0,4.0,2.0,10.0,4.0,0.0,0.0
2,50135262,2007-02-09,0.0,2.0,1.0,0.0,3.0,9.0,5.0,2.0,0.0
3,50135262,2007-02-10,0.0,1.0,0.0,7.0,2.0,2.0,0.0,0.0,0.0
4,50135262,2007-02-11,0.0,1.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0


In [8]:
merged = merged.merge(cpt_wide, how='left', left_on=['ruid', 'hospital_day'], right_on=['ruid', 'event_date'])
merged.shape

(128389, 23)

In [9]:
icd_wide = pd.read_pickle(dir + './icd_wide.pkl')
icd_wide.head()

code_cat,ruid,event_date,dx_blood,dx_circulatory,dx_congenital,dx_digestive,dx_endocrine,dx_external,dx_gu,dx_infection,...,dx_muscskel,dx_neoplasm,dx_nervous,dx_obstetric,dx_perinatal,dx_respiratory,dx_skin,dx_symptoms,proc,visit
0,50135262,1991-06-25,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50135262,2002-02-16,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50135262,2002-11-12,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,50135262,2002-11-17,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50135262,2002-11-20,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
merged = merged.merge(icd_wide, how='left', left_on=['ruid', 'hospital_day'], right_on=['ruid', 'event_date'])
merged.shape

(128389, 44)

In [11]:
med_classes_final_ruids = pd.read_pickle(dir + './med_classes_final_ruids.pkl')
med_classes_final_ruids.head()

,ruid,entry_date,14-alpha Demethylase Inhibitors,5-alpha Reductase Inhibitors,"Abortifacient Agents, Nonsteroidal",Acetaldehyde Dehydrogenase Inhibitors,Acetylcholine Release Inhibitors,Acid Sensing Ion Channel Blockers,Adenosine A2 Receptor Agonists,Adenosine Deaminase Inhibitors,...,Uncoupling Agents,Uricosuric Agents,Urological Agents,Vasoconstrictor Agents,Vasodilator Agents,Viscosupplements,Vitamin B Complex,Vitamins,Voltage-Gated Sodium Channel Blockers,beta-Lactamase Inhibitors
0,50135262,2005-01-09,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,50135262,2007-01-15,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50135262,2015-01-25,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,50135262,2007-02-08,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,50135262,2007-02-09,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
merged = merged.merge(med_classes_final_ruids, 
                      how='left', left_on=['ruid', 'hospital_day'], right_on=['ruid', 'entry_date'])
merged.shape

(128389, 384)

In [13]:
merged.head()

,ruid,visit_id,admit_date,discharge_date,hospital_day,stay_length,n_transfers,readmit_time,readmit_30d,sex,...,Uncoupling Agents,Uricosuric Agents,Urological Agents,Vasoconstrictor Agents,Vasodilator Agents,Viscosupplements,Vitamin B Complex,Vitamins,Voltage-Gated Sodium Channel Blockers,beta-Lactamase Inhibitors
0,50135262,0,2007-02-08,2007-02-12,2007-02-08,4 days,2,NaT,0,F,...,NaN,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,50135262,0,2007-02-08,2007-02-12,2007-02-09,4 days,2,NaT,0,F,...,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50135262,0,2007-02-08,2007-02-12,2007-02-10,4 days,2,NaT,0,F,...,NaN,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
3,50135262,0,2007-02-08,2007-02-12,2007-02-11,4 days,2,NaT,0,F,...,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50135262,0,2007-02-08,2007-02-12,2007-02-12,4 days,2,NaT,0,F,...,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
merged.tail()

,ruid,visit_id,admit_date,discharge_date,hospital_day,stay_length,n_transfers,readmit_time,readmit_30d,sex,...,Uncoupling Agents,Uricosuric Agents,Urological Agents,Vasoconstrictor Agents,Vasodilator Agents,Viscosupplements,Vitamin B Complex,Vitamins,Voltage-Gated Sodium Channel Blockers,beta-Lactamase Inhibitors
128384,53736422,0,2006-04-21,2006-05-05,2006-05-01,14 days,8,NaT,0,M,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
128385,53736422,0,2006-04-21,2006-05-05,2006-05-02,14 days,8,NaT,0,M,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
128386,53736422,0,2006-04-21,2006-05-05,2006-05-03,14 days,8,NaT,0,M,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
128387,53736422,0,2006-04-21,2006-05-05,2006-05-04,14 days,8,NaT,0,M,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
128388,53736422,0,2006-04-21,2006-05-05,2006-05-05,14 days,8,NaT,0,M,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
merged.to_csv(dir + './alvin_merged.csv')